# MIMIC-III (<span style="color:red">ATTENTION : les données dans mimic-iii correspondent à la base de données demo, seulement 100 patients répertoriés. La base entière fait 43 Go</span>)

### Infos sur la dataset : https://physionet.org/content/mimiciii/1.4/, https://mimic.mit.edu/

26 tables... quelles sont celles qui nous intéresse ? description des tables : https://mimic.mit.edu/docs/iii/tables/

### les 2 plus importantes :
- PRESCRIPTIONS.csv : médicaments donnés à chaque patient (date de début+fin d'administration)
- DIAGNOSES_ICD.csv : maladies diagnostiquées (ICD-9 et patient id)

### à savoir : 

- PATIENTS.csv : répertorie les patients, leur genre, date d'admission+sortie
- D_ICD_DIAGNOSES.csv et D_ICD_PROCEDURES.csv : associe le code ICD9 de chaque maladie à son nom (ex:01730 <-> tuberculosis of the eye)
- DRGCODES.csv : symptomes de chaque patient (je crois)
- MICROBIOLOGYEVENTS.csv : analyses biologiques des patients 


In [22]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Exploration de DIAGNOSES_ICD.csv

In [23]:
df = pd.read_csv("mimic-iii-complet/DIAGNOSES_ICD.csv")

In [24]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [25]:
df.shape

(651047, 5)

#### cb de maladies différentes ? 6984 maladies pour 46520 patients

In [27]:
df['ICD9_CODE'].nunique()

6984

In [28]:
df['SUBJECT_ID'].nunique()

46520

#### repartition de ces maladies ?

In [29]:
dp = df['ICD9_CODE'].value_counts()
dp

4019     20703
4280     13111
42731    12891
41401    12429
5849      9119
         ...  
1270         1
E9659        1
71926        1
9593         1
20930        1
Name: ICD9_CODE, Length: 6984, dtype: int64

In [30]:
(dp==1).value_counts()

False    5481
True     1503
Name: ICD9_CODE, dtype: int64

#### -> bcp de ces maladies (1500) n'apparaissent qu'une fois dans la base

In [31]:
(dp<5).value_counts()

False    3874
True     3110
Name: ICD9_CODE, dtype: int64

#### -> la moitié de ces maladies (3100) apparaissent moins de 5 fois dans la base

In [32]:
patients = pd.read_csv("mimic-iii-complet/PATIENTS.csv")

In [34]:
patients["SUBJECT_ID"].nunique()

46520

# ML

https://github.com/MLforHealth/MIMIC_Extract/tree/master/notebooks : Baselines for Mortality and LOS prediction (sklearn)